In [1]:
import glob
import pandas as pd
import numpy as np
from google.colab import drive
import torch
from sklearn.model_selection import train_test_split
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
drive.mount('/content/drive')

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Mounted at /content/drive


In [23]:
import pickle
with open('/content/drive/MyDrive/BioInfo/cols_var_5000.pkl', 'rb') as file:
    cols = pickle.load(file)
cols

array(['ENSG00000274618.2', 'ENSG00000169474.4', 'ENSG00000169469.9', ...,
       'ENSG00000162843.17', 'ENSG00000205517.13', 'ENSG00000184166.2'],
      dtype=object)

In [5]:
links_meth = pd.read_csv('/content/drive/MyDrive/lung/methylation_gene_connections.csv')
links_meth

,gene_name,Name,gene_id
0,A1BG,cg22568540,ENSG00000121410.12
1,A2M,cg12058490,ENSG00000175899.15
2,A2ML1,cg03490200,ENSG00000166535.20
3,A2ML1,cg27653134,ENSG00000166535.20
4,A4GALT,cg07393322,ENSG00000128274.17
...,...,...,...
21429,ZYX,cg03100196,ENSG00000159840.16
21430,ZZEF1,cg05922057,ENSG00000074755.15
21431,ZZEF1,cg07607921,ENSG00000074755.15
21432,ZZZ3,cg04127303,ENSG00000036549.13


In [6]:
links_meth = links_meth[links_meth['gene_id'].isin(cols)]
links_meth

,gene_name,Name,gene_id
2,A2ML1,cg03490200,ENSG00000166535.20
3,A2ML1,cg27653134,ENSG00000166535.20
6,A4GNT,cg17687282,ENSG00000118017.4
7,A4GNT,cg18931888,ENSG00000118017.4
12,AADAC,cg05564657,ENSG00000114771.14
...,...,...,...
21411,ZPBP2,cg14914852,ENSG00000186075.13
21412,ZPLD1,cg01913908,ENSG00000170044.8
21414,ZSCAN1,cg07942995,ENSG00000152467.10
21415,ZSCAN1,cg21331821,ENSG00000152467.10


In [24]:
cols_meth = list(set(links_meth['Name'].values))
len(cols_meth)

4891

#Normal

In [52]:
df_meth_normal = pd.read_csv('/content/drive/MyDrive/lung/meth_normal_id.csv').drop('tumor', axis=1).set_index('Unnamed: 0').fillna(value=0)
df_meth_normal

,cg24499411,cg01050433,cg15195412,cg00616135,cg02547439,cg03469082,cg16007628,cg03231024,cg15705469,cg03328804,...,cg21184495,cg03270167,cg01422136,cg22062239,cg05511752,cg07039113,cg02956093,cg04819539,cg05863612,cg01837574
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
46098b0b-a53e-425a-abc0-73561c447ea1,0.723179,0.055187,0.226328,0.425131,0.048940,0.105522,0.052308,0.072315,0.037979,0.132515,...,0.070181,0.195271,0.0,0.044274,0.050310,0.872874,0.087458,0.927197,0.928514,0.366223
028061ef-a99c-42dd-b011-344036e5357f,0.831239,0.000000,0.677376,0.847075,0.000000,0.000000,0.000000,0.346222,0.063730,0.000000,...,0.000000,0.424517,0.0,0.000000,0.000000,0.866168,0.000000,0.879923,0.698598,0.805365
5d990ed9-53e4-49c5-aeea-ae2ef30128ec,0.751209,0.084713,0.340114,0.525213,0.057883,0.152850,0.056379,0.082196,0.029669,0.130935,...,0.047473,0.150961,0.0,0.046789,0.056862,0.784414,0.099160,0.950257,0.926784,0.406172
a2a050e0-23b8-45ae-888e-010874616996,0.763430,0.110886,0.240634,0.429514,0.064461,0.116867,0.097244,0.066380,0.036419,0.149418,...,0.058349,0.278216,0.0,0.047657,0.061689,0.866339,0.092332,0.944480,0.930388,0.364639
2f791c81-1325-4d3d-aa2f-fb07e80a71b3,0.647552,0.067187,0.281731,0.489314,0.053463,0.127102,0.054755,0.069904,0.031921,0.115911,...,0.042106,0.187335,0.0,0.058542,0.050279,0.785350,0.120934,0.954078,0.927885,0.454263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4ba11f85-5f18-4db9-af0f-c0da0b78f500,0.816210,0.097921,0.368268,0.536518,0.054664,0.137491,0.059337,0.083474,0.038351,0.117707,...,0.074031,0.165786,0.0,0.059637,0.043837,0.863659,0.119402,0.954242,0.874609,0.432255
538c7894-e20a-4369-a076-fac566779894,0.791740,0.108262,0.244384,0.456644,0.039548,0.108124,0.046813,0.055129,0.033524,0.120410,...,0.049736,0.222074,0.0,0.046320,0.514479,0.866088,0.089767,0.948484,0.939764,0.394710
530b32cc-06f7-4a32-bc01-086f9447f9b0,0.643731,0.072327,0.273438,0.466185,0.039660,0.156642,0.070658,0.075260,0.030100,0.074864,...,0.055344,0.223222,0.0,0.059931,0.319514,0.829677,0.098522,0.956989,0.930702,0.322612


In [53]:
a = set(df_meth_normal.columns.values).intersection(set(cols_meth))
df_meth_normal = df_meth_normal.loc[:, list(a)]
df_meth_normal

,cg21718857,cg20199333,cg24621042,cg02812142,cg19325985,cg17133183,cg24426405,cg01401376,cg21289015,cg03969797,...,cg16612699,cg09676788,cg01333011,cg21006686,cg06836849,cg20807545,cg12189880,cg13700897,cg22879515,cg18114671
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
46098b0b-a53e-425a-abc0-73561c447ea1,0.032918,0.877799,0.449299,0.781905,0.244978,0.120320,0.688791,0.070547,0.064168,0.806186,...,0.950542,0.380031,0.397820,0.644192,0.270293,0.170464,0.088313,0.165081,0.021466,0.069342
028061ef-a99c-42dd-b011-344036e5357f,0.000000,0.947283,0.802695,0.776416,0.772211,0.000000,0.754371,0.000000,0.693548,0.751943,...,0.838202,0.000000,0.780075,0.768517,0.659314,0.000000,0.000000,0.000000,0.000000,0.000000
5d990ed9-53e4-49c5-aeea-ae2ef30128ec,0.040686,0.867581,0.483671,0.750360,0.264927,0.000000,0.743878,0.070373,0.064534,0.664425,...,0.929927,0.368886,0.343207,0.680205,0.227259,0.227247,0.081445,0.122317,0.019495,0.089109
a2a050e0-23b8-45ae-888e-010874616996,0.036020,0.876989,0.489557,0.852714,0.231521,0.000000,0.721278,0.093369,0.054680,0.749949,...,0.935310,0.398262,0.337681,0.673657,0.282508,0.149477,0.103204,0.204177,0.018900,0.068027
2f791c81-1325-4d3d-aa2f-fb07e80a71b3,0.025678,0.851830,0.440175,0.804987,0.274788,0.117075,0.688168,0.073433,0.047131,0.772794,...,0.936805,0.292546,0.372805,0.729840,0.274885,0.213967,0.079933,0.140333,0.015354,0.067367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4ba11f85-5f18-4db9-af0f-c0da0b78f500,0.027115,0.892930,0.500866,0.875722,0.234198,0.000000,0.774729,0.079229,0.066550,0.865516,...,0.921188,0.451315,0.387900,0.741222,0.206971,0.223547,0.087514,0.158952,0.014507,0.090141
538c7894-e20a-4369-a076-fac566779894,0.030948,0.901034,0.513526,0.788030,0.245548,0.114474,0.685021,0.092532,0.060786,0.855535,...,0.905792,0.389468,0.369894,0.685565,0.269260,0.173091,0.075350,0.149921,0.015250,0.099004
530b32cc-06f7-4a32-bc01-086f9447f9b0,0.022014,0.886089,0.477139,0.786256,0.261131,0.091629,0.647829,0.156149,0.123854,0.763620,...,0.933119,0.355221,0.387714,0.679363,0.234578,0.213996,0.122895,0.112422,0.021884,0.121863


In [54]:
df_meth_normal['tumor'] = 0

#Tumor

In [55]:
df_meth_tumor = pd.read_csv('/content/drive/MyDrive/lung/meth_tumor_id.csv').drop('tumor', axis=1).set_index('Unnamed: 0').fillna(value=0)
df_meth_tumor

,cg02932167,cg23594876,cg11903096,cg13842648,cg01505590,cg00571634,cg18611281,cg24682149,cg03383661,cg27560864,...,cg21846488,cg18794577,cg06059810,cg15952487,cg21301148,cg14303674,cg16837769,cg24217726,cg06394897,cg05358404
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
56c07b06-c6d3-4c03-9e57-7be636e7cc5c,0.837288,0.046871,0.081712,0.461749,0.015053,0.044319,0.119393,0.069918,0.064286,0.319976,...,0.571640,0.042933,0.880821,0.227056,0.203054,0.000000,0.027856,0.929990,0.0,0.692485
56c07b06-c6d3-4c03-9e57-7be636e7cc5c,0.906502,0.033266,0.062061,0.529717,0.012732,0.040466,0.113311,0.070466,0.068304,0.420652,...,0.472376,0.032852,0.889434,0.059717,0.095326,0.766781,0.021747,0.934113,0.0,0.686041
d55de89b-c5bd-4901-9de9-fecaae0f9cfd,0.626647,0.057083,0.052108,0.462933,0.013114,0.047103,0.079947,0.080009,0.066904,0.343533,...,0.437849,0.214133,0.931682,0.568111,0.313224,0.536870,0.030420,0.944846,0.0,0.637081
c6e1b9d8-09c8-4e05-ba87-ca985db813e3,0.775840,0.049144,0.042735,0.526428,0.010891,0.038711,0.047245,0.051898,0.054296,0.565719,...,0.581426,0.026804,0.749117,0.557031,0.260284,0.694623,0.034649,0.937814,0.0,0.654207
217bbeb4-6b93-47d9-808f-14fd17e9b3b4,0.734258,0.039724,0.052968,0.744504,0.015894,0.037222,0.148537,0.073689,0.064726,0.333631,...,0.637343,0.033501,0.848108,0.517427,0.424802,0.366125,0.030665,0.917408,0.0,0.705675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54af5967-283d-4dd0-90e0-d2c351694c4c,0.529446,0.038834,0.057724,0.393793,0.012164,0.041384,0.063126,0.079525,0.067876,0.433770,...,0.323323,0.060434,0.890379,0.429536,0.240575,0.000000,0.030379,0.911825,0.0,0.761498
570d34b4-1b20-4124-9086-5e4f678ad64d,0.657467,0.038391,0.053877,0.739595,0.018473,0.047011,0.098292,0.074264,0.082431,0.505654,...,0.212189,0.077722,0.904043,0.282490,0.134611,0.641344,0.038581,0.942086,0.0,0.835342
5d9f9c9d-2188-4106-b75b-6add5e560855,0.742539,0.046385,0.037669,0.631300,0.012132,0.033185,0.061682,0.065815,0.056262,0.401196,...,0.669181,0.381238,0.839367,0.581529,0.308806,0.455742,0.029296,0.937663,0.0,0.705104


In [56]:
a = set(df_meth_tumor.columns.values).intersection(set(cols_meth))
df_meth_tumor = df_meth_tumor.loc[:, list(a)]
df_meth_tumor

,cg21718857,cg20199333,cg24621042,cg02812142,cg19325985,cg17133183,cg24426405,cg01401376,cg21289015,cg03969797,...,cg16612699,cg09676788,cg01333011,cg21006686,cg06836849,cg20807545,cg12189880,cg13700897,cg22879515,cg18114671
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
56c07b06-c6d3-4c03-9e57-7be636e7cc5c,0.031469,0.601895,0.345314,0.461699,0.601514,0.000000,0.312814,0.091638,0.071820,0.236310,...,0.509615,0.210199,0.261708,0.914262,0.251416,0.146953,0.081654,0.130021,0.310344,0.085494
56c07b06-c6d3-4c03-9e57-7be636e7cc5c,0.116463,0.676961,0.402927,0.243169,0.967771,0.000000,0.106304,0.056050,0.067464,0.052634,...,0.165021,0.481109,0.945519,0.949788,0.594061,0.043869,0.075589,0.152512,0.182979,0.111970
d55de89b-c5bd-4901-9de9-fecaae0f9cfd,0.034212,0.836848,0.447151,0.698549,0.327953,0.153025,0.559027,0.081530,0.091214,0.385010,...,0.635689,0.439650,0.344083,0.531495,0.111697,0.368900,0.068862,0.081817,0.174268,0.128662
c6e1b9d8-09c8-4e05-ba87-ca985db813e3,0.018299,0.855466,0.428447,0.722100,0.349816,0.000000,0.429433,0.524028,0.042004,0.550279,...,0.854431,0.000000,0.487122,0.490820,0.202423,0.333200,0.076620,0.120625,0.014402,0.085397
217bbeb4-6b93-47d9-808f-14fd17e9b3b4,0.333955,0.860077,0.614284,0.742325,0.549470,0.120044,0.262325,0.062239,0.038566,0.313631,...,0.867681,0.333728,0.370598,0.745246,0.404688,0.354546,0.066848,0.118754,0.406241,0.155220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54af5967-283d-4dd0-90e0-d2c351694c4c,0.025624,0.619454,0.356972,0.621862,0.228688,0.106247,0.464693,0.286112,0.000000,0.493602,...,0.611057,0.236917,0.307525,0.317073,0.117171,0.248594,0.079556,0.129994,0.016024,0.124320
570d34b4-1b20-4124-9086-5e4f678ad64d,0.041465,0.899818,0.230308,0.353691,0.523112,0.107069,0.241254,0.692762,0.166084,0.232100,...,0.400036,0.251945,0.308407,0.319162,0.543312,0.131254,0.067184,0.150541,0.054951,0.093534
5d9f9c9d-2188-4106-b75b-6add5e560855,0.025742,0.856768,0.462233,0.638402,0.403508,0.000000,0.394149,0.485939,0.000000,0.758376,...,0.942642,0.347256,0.332062,0.637192,0.211514,0.305494,0.075205,0.091619,0.016767,0.214968


In [57]:
df_meth_tumor['tumor'] = 1

#Merge

In [58]:
a = set(df_meth_normal.columns.values)
b = set(df_meth_tumor.columns.values)
cols = list(a.intersection(b))
len(cols)

4236

In [59]:
met = pd.concat([df_meth_normal.loc[:, cols], df_meth_tumor.loc[:, cols]], axis=0, ignore_index=True)
met

,cg21718857,cg20199333,cg24621042,cg02812142,cg19325985,cg17133183,cg24426405,cg01401376,cg21289015,cg03969797,...,cg16612699,cg09676788,cg01333011,cg21006686,cg06836849,cg20807545,cg12189880,cg13700897,cg22879515,cg18114671
0,0.032918,0.877799,0.449299,0.781905,0.244978,0.120320,0.688791,0.070547,0.064168,0.806186,...,0.950542,0.380031,0.397820,0.644192,0.270293,0.170464,0.088313,0.165081,0.021466,0.069342
1,0.000000,0.947283,0.802695,0.776416,0.772211,0.000000,0.754371,0.000000,0.693548,0.751943,...,0.838202,0.000000,0.780075,0.768517,0.659314,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.040686,0.867581,0.483671,0.750360,0.264927,0.000000,0.743878,0.070373,0.064534,0.664425,...,0.929927,0.368886,0.343207,0.680205,0.227259,0.227247,0.081445,0.122317,0.019495,0.089109
3,0.036020,0.876989,0.489557,0.852714,0.231521,0.000000,0.721278,0.093369,0.054680,0.749949,...,0.935310,0.398262,0.337681,0.673657,0.282508,0.149477,0.103204,0.204177,0.018900,0.068027
4,0.025678,0.851830,0.440175,0.804987,0.274788,0.117075,0.688168,0.073433,0.047131,0.772794,...,0.936805,0.292546,0.372805,0.729840,0.274885,0.213967,0.079933,0.140333,0.015354,0.067367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707,0.025624,0.619454,0.356972,0.621862,0.228688,0.106247,0.464693,0.286112,0.000000,0.493602,...,0.611057,0.236917,0.307525,0.317073,0.117171,0.248594,0.079556,0.129994,0.016024,0.124320
1708,0.041465,0.899818,0.230308,0.353691,0.523112,0.107069,0.241254,0.692762,0.166084,0.232100,...,0.400036,0.251945,0.308407,0.319162,0.543312,0.131254,0.067184,0.150541,0.054951,0.093534
1709,0.025742,0.856768,0.462233,0.638402,0.403508,0.000000,0.394149,0.485939,0.000000,0.758376,...,0.942642,0.347256,0.332062,0.637192,0.211514,0.305494,0.075205,0.091619,0.016767,0.214968
1710,0.024837,0.838172,0.437883,0.653634,0.340608,0.117334,0.615699,0.107249,0.000000,0.577505,...,0.824865,0.426063,0.341619,0.572106,0.167290,0.374015,0.078858,0.122126,0.035137,0.131777


In [60]:
#shuffle
met = met.sample(frac=1).reset_index(drop=True)
met

,cg21718857,cg20199333,cg24621042,cg02812142,cg19325985,cg17133183,cg24426405,cg01401376,cg21289015,cg03969797,...,cg16612699,cg09676788,cg01333011,cg21006686,cg06836849,cg20807545,cg12189880,cg13700897,cg22879515,cg18114671
0,0.000000,0.102658,0.000000,0.161916,0.000000,0.000000,0.125074,0.000000,0.681719,0.061606,...,0.000000,0.088814,0.171343,0.000000,0.000000,0.000000,0.192164,0.000000,0.482049,0.000000
1,0.024299,0.886675,0.636840,0.500554,0.827699,0.056631,0.164649,0.058080,0.034160,0.000000,...,0.322193,0.130541,0.260433,0.904227,0.178612,0.203707,0.080744,0.128860,0.563722,0.671391
2,0.032883,0.849967,0.362268,0.440021,0.413430,0.723352,0.497043,0.132740,0.462162,0.000000,...,0.653064,0.000000,0.503073,0.578552,0.211435,0.436091,0.074718,0.243203,0.499401,0.315835
3,0.047148,0.833412,0.417303,0.242822,0.256190,0.143918,0.821049,0.091616,0.095588,0.830453,...,0.902640,0.521700,0.305871,0.666611,0.231036,0.180822,0.174085,0.000000,0.036559,0.098930
4,0.021704,0.903923,0.477768,0.844462,0.601183,0.059259,0.219711,0.093403,0.035109,0.000000,...,0.488996,0.656240,0.261336,0.641936,0.533235,0.215919,0.060339,0.055992,0.015395,0.060104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707,0.020964,0.894778,0.582306,0.888463,0.737089,0.059199,0.145325,0.074466,0.072515,0.000000,...,0.923539,0.490077,0.745878,0.793390,0.043813,0.525674,0.069090,0.103645,0.013848,0.090537
1708,0.569535,0.923535,0.691379,0.808147,0.722074,0.326344,0.104365,0.057597,0.093475,0.000000,...,0.875093,0.889036,0.217181,0.951171,0.444046,0.731328,0.068983,0.048419,0.676954,0.131511
1709,0.029605,0.916779,0.412740,0.442993,0.525618,0.520012,0.332122,0.382402,0.042940,0.000000,...,0.725373,0.239972,0.412393,0.728980,0.446460,0.455313,0.069243,0.472645,0.018944,0.470106
1710,0.220020,0.877739,0.315482,0.431679,0.512888,0.629362,0.375544,0.650788,0.749463,0.000000,...,0.501270,0.203712,0.449226,0.470842,0.261685,0.542686,0.088212,0.256689,0.018647,0.387100


In [61]:
y = met['tumor']
X = met.drop('tumor', axis=1)

In [19]:
from sklearn.model_selection import train_test_split

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#SVM and other models

In [21]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [64]:
models = {
          'Decision Tree': DecisionTreeClassifier(),
          'KNeighbors Classifier': KNeighborsClassifier(),
          'SVM': svm.SVC(),
          'Random Forest': RandomForestClassifier(max_depth=2, random_state=0),
          'Dummy Classifier': DummyClassifier(strategy="stratified")
          }

def test_model(name, model, traindata, testdata, trainlabels, testlabels):
  model.fit(traindata, trainlabels)
  trainpred = model.predict(traindata)
  testpred = model.predict(testdata)
  print(f" ===== {name} =====")
  print(f"Accuracy on train {accuracy_score(trainlabels, trainpred)}")
  print(f"Accuracy on test {accuracy_score(testlabels, testpred)}")
  print("Confusion matrix on test")
  print(confusion_matrix(testlabels, testpred))

for name, model in models.items():
  test_model(name, model, X_train, X_test, y_train, y_test)

 ===== Decision Tree =====
Accuracy on train 1.0
Accuracy on test 0.967930029154519
Confusion matrix on test
[[ 62   8]
 [  3 270]]
 ===== KNeighbors Classifier =====
Accuracy on train 0.9802775748721695
Accuracy on test 0.9620991253644315
Confusion matrix on test
[[ 66   4]
 [  9 264]]
 ===== SVM =====
Accuracy on train 0.9956172388604821
Accuracy on test 0.9912536443148688
Confusion matrix on test
[[ 67   3]
 [  0 273]]
 ===== Random Forest =====
Accuracy on train 0.9963476990504018
Accuracy on test 0.9825072886297376
Confusion matrix on test
[[ 64   6]
 [  0 273]]
 ===== Dummy Classifier =====
Accuracy on train 0.6990504017531045
Accuracy on test 0.6705539358600583
Confusion matrix on test
[[ 11  59]
 [ 54 219]]


#Tumor/Normal classification

In [65]:
print("TRAIN SHAPE: ", X_train.shape)
print("TEST SHAPE: ", X_test.shape)
print("NORMAL TRAIN: ", y_train[y_train==0].shape)
print("NORMAL Test: ", y_test[y_test==0].shape)

TRAIN SHAPE:  (1369, 4235)
TEST SHAPE:  (343, 4235)
NORMAL TRAIN:  (253,)
NORMAL Test:  (70,)


In [66]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [76]:
import torch
from torch.nn import Linear
from torch_geometric.nn import GCNConv

features_in = X_train.shape[1]
n_classes = len(y_train.unique())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        torch.manual_seed(1234)
        self.conv1 = GCNConv(features_in, 128)
        self.conv2 = GCNConv(128, 64)
        self.conv3 = GCNConv(64, 2)
        self.classifier = Linear(2, n_classes)

    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index)
        h = h.tanh()
        h = self.conv2(h, edge_index)
        h = h.tanh()
        h = self.conv3(h, edge_index)
        h = h.tanh()

        out = self.classifier(h)

        return out

model = GCN().double().to(device)
print(model)

GCN(
  (conv1): GCNConv(4235, 128)
  (conv2): GCNConv(128, 64)
  (conv3): GCNConv(64, 2)
  (classifier): Linear(in_features=2, out_features=2, bias=True)
)


In [68]:
#BUILD CORRELATION MATRIXES
correlation_matrix_train = np.corrcoef(X_train)
correlation_matrix_test = np.corrcoef(X_test)
print("CORR_MATRIX TRAIN shape: ", correlation_matrix_train.shape)
print("CORR_MATRIX TEST shape: ", correlation_matrix_test.shape)

CORR_MATRIX TRAIN shape:  (1369, 1369)
CORR_MATRIX TEST shape:  (343, 343)


In [69]:
#CONVERT ALL TO TENSORS
x_train = torch.from_numpy(X_train.values)
y_train = torch.from_numpy(y_train.values)
x_test = torch.from_numpy(X_test.values)
y_test = torch.from_numpy(y_test.values)

In [79]:
threshold_edge = 0.98 #threshold to determine if two nodes are connected based on the correlation matrix

In [80]:
#BUILD EDGE INDEX FOR TRAIN SET
mask = correlation_matrix_train > threshold_edge

#Exclude the diagonal elements (assuming the matrix is square)
np.fill_diagonal(mask, False)

#Get the row and column indices where the mask is True
row_indices, col_indices = np.where(mask)

#Create the edge_index tensor
edge_index_train = torch.tensor([row_indices, col_indices], dtype=torch.int64)

print(edge_index_train.shape)

torch.Size([2, 2456])


In [81]:
#BUILD EDGE INDEX FOR TEST SET
mask = correlation_matrix_test > threshold_edge

#Exclude the diagonal elements (assuming the matrix is square)
np.fill_diagonal(mask, False)

#Get the row and column indices where the mask is True
row_indices, col_indices = np.where(mask)

#Create the edge_index tensor
edge_index_test = torch.tensor([row_indices, col_indices], dtype=torch.int64)

print(edge_index_test.shape)

torch.Size([2, 100])


In [82]:
x_train = x_train.double().to(device)
edge_index_train = edge_index_train.to(device)
y_train = y_train.to(device)
x_test = x_test.double().to(device)
edge_index_test = edge_index_test.to(device)
y_test = y_test.to(device)

In [83]:
#TRAIN SETTINGS
import torch.optim as optim
import torch.nn as nn

#estimator = GCNEstimator(edge_index=edge_index_train, n_classes=2)
#estimator.fit(x_train, y_train)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 250

In [84]:
import numpy as np

#TRAIN LOOP
for epoch in range(n_epochs):
    optimizer.zero_grad()
    out = model(x_train.double(), edge_index_train)
    loss = loss_fn(out, y_train)
    # backward pass
    loss.backward()
    if epoch % 10 == 0:
      print(f"EPOCH {epoch}, LOSS: {loss}")
    # update weights
    optimizer.step()

EPOCH 0, LOSS: 0.4785742911004868
EPOCH 10, LOSS: 0.4775270385865105
EPOCH 20, LOSS: 0.4653372400273
EPOCH 30, LOSS: 0.29574031092988895
EPOCH 40, LOSS: 0.23294881910845605
EPOCH 50, LOSS: 0.21424117276866878
EPOCH 60, LOSS: 0.20161795482716588
EPOCH 70, LOSS: 0.1910544299737039
EPOCH 80, LOSS: 0.18243406784920455
EPOCH 90, LOSS: 0.17482268144966798
EPOCH 100, LOSS: 0.16796595377386653
EPOCH 110, LOSS: 0.16150985525733697
EPOCH 120, LOSS: 0.15549412771609017
EPOCH 130, LOSS: 0.15006316209414264
EPOCH 140, LOSS: 0.1450680150405239
EPOCH 150, LOSS: 0.14040989755427938
EPOCH 160, LOSS: 0.13604455980654248
EPOCH 170, LOSS: 0.13193612605409077
EPOCH 180, LOSS: 0.12805859082008988
EPOCH 190, LOSS: 0.12438967458424184
EPOCH 200, LOSS: 0.12091037867901627
EPOCH 210, LOSS: 0.11760435519455885
EPOCH 220, LOSS: 0.11445720566608224
EPOCH 230, LOSS: 0.11145620977539336
EPOCH 240, LOSS: 0.10859003087259612


In [85]:
#TEST
model.eval()
out = model(x_test, edge_index_test)
pred = out.argmax(dim=1)  #Use the class with highest probability.
print(pred)
print(y_test)
test_correct = pred == y_test
test_acc = int(test_correct.sum()) / len(test_correct)
print("Accuracy on test set: {}".format(test_acc))

tensor([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
        0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0,
        1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,

#Evaluating only most expressed genes

In [ ]:
connections = pd.read_csv('/content/drive/MyDrive/lung/methylation_gene_connections.csv')
connections

,gene_name,Name,gene_id
0,A1BG,cg22568540,ENSG00000121410.12
1,A2M,cg12058490,ENSG00000175899.15
2,A2ML1,cg03490200,ENSG00000166535.20
3,A2ML1,cg27653134,ENSG00000166535.20
4,A4GALT,cg07393322,ENSG00000128274.17
...,...,...,...
21429,ZYX,cg03100196,ENSG00000159840.16
21430,ZZEF1,cg05922057,ENSG00000074755.15
21431,ZZEF1,cg07607921,ENSG00000074755.15
21432,ZZZ3,cg04127303,ENSG00000036549.13


In [ ]:
most_expressed = pd.read_csv('/content/drive/MyDrive/lung/most_expressed.csv')
most_expressed

,gene_name,node_id
0,ENSG00000169469.9,12385
1,ENSG00000170373.8,12580
2,ENSG00000168530.16,12175
3,ENSG00000274618.2,19383
4,ENSG00000168484.12,12160
...,...,...
7499,ENSG00000184828.10,15157
7500,ENSG00000203747.12,17010
7501,ENSG00000135318.12,6955
7502,ENSG00000203722.8,17002


In [ ]:
a = set(most_expressed['gene_name'])
b = set(connections['gene_id'])
len(a.intersection(b))

4121

In [ ]:
join = most_expressed.set_index('gene_name').join(connections.set_index('gene_id'), how='inner')
join

,node_id,gene_name,Name
ENSG00000000005.6,1,TNMD,cg00403724
ENSG00000000938.13,5,FGR,cg06827976
ENSG00000000938.13,5,FGR,cg11254522
ENSG00000001084.13,8,GCLC,cg08302713
ENSG00000001084.13,8,GCLC,cg12013321
...,...,...,...
ENSG00000284194.3,19687,SCO2,cg15018760
ENSG00000284194.3,19687,SCO2,cg24856732
ENSG00000284770.2,19739,TBCE,cg18036707
ENSG00000288611.1,19932,NPBWR1,cg02115041


In [ ]:
len(set(join.index.values))

4121

In [ ]:
islands = set(join['Name'])
len(islands)

7850

In [ ]:
islands.add('tumor')
len(islands)

7851

In [ ]:
met_cols = set(met.columns.values)
common_cols = met_cols.intersection(islands)
len(common_cols)

6801

In [ ]:
met_expressed = met.loc[:, common_cols]
met_expressed

<ipython-input-79-118392a4c6f2>:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  met_expressed = met.loc[:, common_cols]


,cg13436799,cg19815720,cg09827833,cg15876417,cg12331389,cg20092728,cg17733331,cg06882926,cg04481779,cg19781251,...,cg17693270,cg04473302,cg15626350,cg03580247,cg21366834,cg24687051,cg25157408,cg03310469,cg23142900,cg11822659
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.036395,0.666086,0.284552,0.066883,0.018768,0.789669,0.093192,0.736185,0.336655,0.960421,...,0.390390,0.029203,0.400031,0.903389,0.915298,0.069189,0.941056,0.069403,0.026789,0.069682
2,0.034846,0.702559,0.553147,0.135686,0.048781,0.765321,0.089256,0.577267,0.289119,0.958780,...,0.334737,0.186365,0.836073,0.833238,0.948430,0.051669,0.949494,0.098806,0.036695,0.347090
3,0.032254,0.722512,0.206091,0.070017,0.015381,0.842113,0.074919,0.787001,0.398892,0.944200,...,0.391182,0.029655,0.437829,0.946007,0.923634,0.044936,0.934616,0.043135,0.023292,0.085345
4,0.032257,0.627071,0.258535,0.048716,0.026104,0.762670,0.114807,0.748254,0.363123,0.967436,...,0.313817,0.030581,0.415810,0.922631,0.933144,0.061749,0.956726,0.078607,0.032538,0.068183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.032788,0.731666,0.146950,0.083108,0.048755,0.694756,0.066790,0.273214,0.157050,0.943729,...,0.465538,0.052587,0.394058,0.551849,0.947440,0.050224,0.800296,0.098368,0.035690,0.463048
158,0.032639,0.436790,0.612608,0.189598,0.018567,0.892294,0.080052,0.193941,0.084523,0.968753,...,0.258781,0.028624,0.111339,0.222674,0.947531,0.054326,0.680698,0.055243,0.031464,0.079972
159,0.044083,0.739952,0.269540,0.063305,0.029817,0.827901,0.089686,0.767726,0.352851,0.959638,...,0.502602,0.039763,0.296246,0.871860,0.908371,0.049264,0.956616,0.087865,0.029009,0.084162
160,0.047259,0.550509,0.266219,0.040946,0.033694,0.854092,0.123378,0.587950,0.408165,0.965944,...,0.308737,0.053975,0.509971,0.901873,0.946546,0.077311,0.929667,0.058218,0.031659,0.068837


In [ ]:
'tumor' in met_expressed.columns.values

True

In [ ]:
X = met_expressed.drop('tumor', axis=1)
y = met_expressed['tumor']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
print("TRAIN SHAPE: ", X_train.shape)
print("TEST SHAPE: ", X_test.shape)
print("NORMAL TRAIN: ", y_train[y_train==0].shape)
print("NORMAL Test: ", y_test[y_test==0].shape)

TRAIN SHAPE:  (129, 6800)
TEST SHAPE:  (33, 6800)
NORMAL TRAIN:  (65,)
NORMAL Test:  (16,)


In [ ]:
features_in = X_train.shape[1]
n_classes = len(y_train.unique())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        torch.manual_seed(1234)
        self.conv1 = GCNConv(features_in, 512)
        self.conv2 = GCNConv(512, 256)
        self.conv3 = GCNConv(256, 2)
        self.classifier = Linear(2, n_classes)

    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index)
        h = h.tanh()
        h = self.conv2(h, edge_index)
        h = h.tanh()
        h = self.conv3(h, edge_index)
        h = h.tanh()

        out = self.classifier(h)

        return out

model = GCN().double().to(device)
print(model)

GCN(
  (conv1): GCNConv(6800, 512)
  (conv2): GCNConv(512, 256)
  (conv3): GCNConv(256, 2)
  (classifier): Linear(in_features=2, out_features=2, bias=True)
)


In [ ]:
#BUILD CORRELATION MATRIXES
correlation_matrix_train = np.corrcoef(X_train)
correlation_matrix_test = np.corrcoef(X_test)
print("CORR_MATRIX TRAIN shape: ", correlation_matrix_train.shape)
print("CORR_MATRIX TEST shape: ", correlation_matrix_test.shape)

CORR_MATRIX TRAIN shape:  (129, 129)
CORR_MATRIX TEST shape:  (33, 33)


/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2853: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [ ]:
#CONVERT ALL TO TENSORS
x_train = torch.from_numpy(X_train.values)
y_train = torch.from_numpy(y_train.values)
x_test = torch.from_numpy(X_test.values)
y_test = torch.from_numpy(y_test.values)

In [ ]:
threshold_edge = 0.95 #threshold to determine if two nodes are connected based on the correlation matrix

In [ ]:
#BUILD EDGE INDEX FOR TRAIN SET
mask = correlation_matrix_train > threshold_edge

#Exclude the diagonal elements (assuming the matrix is square)
np.fill_diagonal(mask, False)

#Get the row and column indices where the mask is True
row_indices, col_indices = np.where(mask)

#Create the edge_index tensor
edge_index_train = torch.tensor([row_indices, col_indices], dtype=torch.int64)

print(edge_index_train.shape)

torch.Size([2, 3678])


In [ ]:
#BUILD EDGE INDEX FOR TEST SET
mask = correlation_matrix_test > threshold_edge

#Exclude the diagonal elements (assuming the matrix is square)
np.fill_diagonal(mask, False)

#Get the row and column indices where the mask is True
row_indices, col_indices = np.where(mask)

#Create the edge_index tensor
edge_index_test = torch.tensor([row_indices, col_indices], dtype=torch.int64)

print(edge_index_test.shape)

torch.Size([2, 198])


In [ ]:
x_train = x_train.double().to(device)
edge_index_train = edge_index_train.to(device)
y_train = y_train.to(device)
x_test = x_test.double().to(device)
edge_index_test = edge_index_test.to(device)
y_test = y_test.to(device)

In [ ]:
#TRAIN SETTINGS
import torch.optim as optim
import torch.nn as nn

#estimator = GCNEstimator(edge_index=edge_index_train, n_classes=2)
#estimator.fit(x_train, y_train)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 500

In [ ]:
import numpy as np

#TRAIN LOOP
for epoch in range(n_epochs):
    optimizer.zero_grad()
    out = model(x_train.double(), edge_index_train)
    loss = loss_fn(out, y_train)
    # backward pass
    loss.backward()
    if epoch % 10 == 0:
      print(f"EPOCH {epoch}, LOSS: {loss}")
    # update weights
    optimizer.step()

EPOCH 0, LOSS: 0.6790390770937312
EPOCH 10, LOSS: 0.5142242359301292
EPOCH 20, LOSS: 0.46064560615153277
EPOCH 30, LOSS: 0.4768002118250038
EPOCH 40, LOSS: 0.4473772005647153
EPOCH 50, LOSS: 0.4431621974622731
EPOCH 60, LOSS: 0.4288690610381512
EPOCH 70, LOSS: 0.4048742898032767
EPOCH 80, LOSS: 0.39610224212682726
EPOCH 90, LOSS: 0.38670634895455613
EPOCH 100, LOSS: 0.3786149835904561
EPOCH 110, LOSS: 0.3074930644610033
EPOCH 120, LOSS: 0.2889139899750882
EPOCH 130, LOSS: 0.2748062813798453
EPOCH 140, LOSS: 0.2993122223310735
EPOCH 150, LOSS: 0.25189396899585426
EPOCH 160, LOSS: 0.24207187825599322
EPOCH 170, LOSS: 0.23239525032434566
EPOCH 180, LOSS: 0.22398861561226127
EPOCH 190, LOSS: 0.2133972750006474
EPOCH 200, LOSS: 0.20497666120247582
EPOCH 210, LOSS: 0.19863789055116526
EPOCH 220, LOSS: 0.19139213588710444
EPOCH 230, LOSS: 0.18550452674397108
EPOCH 240, LOSS: 0.1799931511344079
EPOCH 250, LOSS: 0.17487667597684467
EPOCH 260, LOSS: 0.1716030522332003
EPOCH 270, LOSS: 0.18164247

In [ ]:
#TEST
model.eval()
out = model(x_test, edge_index_test)
pred = out.argmax(dim=1)  #Use the class with highest probability.
print(pred)
print(y_test)
test_correct = pred == y_test
test_acc = int(test_correct.sum()) / len(test_correct)
print("Accuracy on test set: {}".format(test_acc))

tensor([1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 0, 1, 0])
tensor([1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 0, 1, 0])
Accuracy on test set: 0.9696969696969697


#Evaluating some most and some least expressed genes

In [ ]:
connections = pd.read_csv('/content/drive/MyDrive/lung/methylation_gene_connections.csv')
connections

,gene_name,Name,gene_id
0,A1BG,cg22568540,ENSG00000121410.12
1,A2M,cg12058490,ENSG00000175899.15
2,A2ML1,cg03490200,ENSG00000166535.20
3,A2ML1,cg27653134,ENSG00000166535.20
4,A4GALT,cg07393322,ENSG00000128274.17
...,...,...,...
21429,ZYX,cg03100196,ENSG00000159840.16
21430,ZZEF1,cg05922057,ENSG00000074755.15
21431,ZZEF1,cg07607921,ENSG00000074755.15
21432,ZZZ3,cg04127303,ENSG00000036549.13


In [ ]:
most_expressed = pd.read_csv('/content/drive/MyDrive/lung/most_expressed.csv')
most_expressed

,gene_name,node_id
0,ENSG00000169469.9,12385
1,ENSG00000170373.8,12580
2,ENSG00000168530.16,12175
3,ENSG00000274618.2,19383
4,ENSG00000168484.12,12160
...,...,...
7499,ENSG00000184828.10,15157
7500,ENSG00000203747.12,17010
7501,ENSG00000135318.12,6955
7502,ENSG00000203722.8,17002


In [ ]:
least_expressed = pd.read_csv('/content/drive/MyDrive/lung/least_expressed.csv')
least_expressed

,gene_name
0,ENSG00000197497.11
1,ENSG00000196735.13
2,ENSG00000106633.17
3,ENSG00000129911.9
4,ENSG00000196860.8
...,...
12453,ENSG00000283977.1
12454,ENSG00000283967.1
12455,ENSG00000283952.1
12456,ENSG00000256453.2


In [ ]:
cols = np.concatenate((most_expressed[:1024]['gene_name'].values, least_expressed[-1024:]['gene_name'].values))
len(cols)

2048

In [ ]:
a = set(cols)
b = set(connections['gene_id'])
cols_final = a.intersection(b)
len(cols_final)

789

In [ ]:
conn = connections[connections['gene_id'].isin(cols_final)]
conn

,gene_name,Name,gene_id
13,AADACL2,cg25159668,ENSG00000197953.6
45,ABCA8,cg21660392,ENSG00000141338.14
123,ABI1,cg17918201,ENSG00000136754.18
124,ABI1,cg23087130,ENSG00000136754.18
159,ACADL,cg09068528,ENSG00000115361.8
...,...,...,...
21022,ZNF263,cg17171916,ENSG00000006194.10
21023,ZNF263,cg23412875,ENSG00000006194.10
21055,ZNF317,cg05357320,ENSG00000130803.15
21268,ZNF593,cg11418559,ENSG00000142684.9


In [ ]:
a = set(conn['Name'].values)
b = set(met.columns.values)
c = a.intersection(b)
len(c)

1219

In [ ]:
c.add('tumor')
met_final = met.loc[:, c]
met_final

<ipython-input-15-2c8fdf53d005>:2: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  met_final = met.loc[:, c]


,cg17503456,cg01714932,cg12370791,cg11442050,cg21274570,cg19823847,cg25995916,cg08658594,cg26845278,cg22093158,...,cg19149785,cg12020639,cg23112213,cg11200929,cg25283338,cg19384697,cg04595372,cg12592716,cg21376883,cg17703212
0,0.586412,0.872750,0.168477,0.033928,0.918639,0.935316,0.053531,0.965513,0.903157,0.042357,...,0.853095,0.043923,0.098106,0.845239,0.043505,0.210515,0.460062,0.032910,0.548352,0.530375
1,0.085731,0.928048,0.133277,0.031930,0.750490,0.830055,0.078509,0.927001,0.857652,0.065099,...,0.576435,0.051238,0.183732,0.130570,0.033428,0.332792,0.478744,0.034727,0.205394,0.353882
2,0.706683,0.629485,0.627415,0.058957,0.901168,0.929764,0.073447,0.920519,0.907020,0.057838,...,0.608593,0.039954,0.121861,0.672395,0.033105,0.314555,0.732580,0.038261,0.559648,0.725051
3,0.074202,0.502972,0.111327,0.033354,0.375206,0.698388,0.076652,0.950662,0.706692,0.056575,...,0.817267,0.036883,0.000000,0.091512,0.032019,0.196255,0.291341,0.128707,0.141301,0.223726
4,0.096323,0.892434,0.193149,0.064041,0.637744,0.814121,0.079324,0.903503,0.840162,0.038488,...,0.675660,0.068010,0.132096,0.139083,0.040316,0.282313,0.443025,0.029563,0.218551,0.319665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.186792,0.922671,0.101463,0.079819,0.905837,0.803162,0.095897,0.933743,0.895978,0.090674,...,0.785403,0.076334,0.122911,0.355847,0.060519,0.305503,0.611377,0.110740,0.335503,0.407963
158,0.640156,0.944036,0.052023,0.051113,0.946276,0.372576,0.059122,0.948217,0.874138,0.037772,...,0.829278,0.021646,0.144263,0.363263,0.031826,0.218616,0.425956,0.400766,0.405899,0.216625
159,0.122059,0.948534,0.109939,0.033120,0.881140,0.857787,0.049359,0.940743,0.914561,0.058220,...,0.716248,0.039779,0.153233,0.183160,0.057735,0.258257,0.532127,0.081556,0.477970,0.211607
160,0.107398,0.870075,0.087115,0.039592,0.770142,0.558243,0.061546,0.864514,0.702779,0.112768,...,0.691544,0.038977,0.120822,0.282009,0.028081,0.359713,0.608913,0.129365,0.356941,0.394346


In [ ]:
'tumor' in met_final.columns.values

True

In [ ]:
X = met_final.drop('tumor', axis=1)
y = met_final['tumor']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
print("TRAIN SHAPE: ", X_train.shape)
print("TEST SHAPE: ", X_test.shape)
print("NORMAL TRAIN: ", y_train[y_train==0].shape)
print("NORMAL Test: ", y_test[y_test==0].shape)

TRAIN SHAPE:  (129, 1219)
TEST SHAPE:  (33, 1219)
NORMAL TRAIN:  (67,)
NORMAL Test:  (14,)


In [ ]:
from torch.nn import Linear
from torch_geometric.nn import GCNConv

features_in = X_train.shape[1]
n_classes = len(y_train.unique())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        torch.manual_seed(1234)
        self.conv1 = GCNConv(features_in, 512)
        self.conv2 = GCNConv(512, 256)
        self.conv3 = GCNConv(256, 2)
        self.classifier = Linear(2, n_classes)

    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index)
        h = h.tanh()
        h = self.conv2(h, edge_index)
        h = h.tanh()
        h = self.conv3(h, edge_index)
        h = h.tanh()

        out = self.classifier(h)

        return out

model = GCN().double().to(device)
print(model)

GCN(
  (conv1): GCNConv(1219, 512)
  (conv2): GCNConv(512, 256)
  (conv3): GCNConv(256, 2)
  (classifier): Linear(in_features=2, out_features=2, bias=True)
)


In [ ]:
#BUILD CORRELATION MATRIXES
correlation_matrix_train = np.corrcoef(X_train)
correlation_matrix_test = np.corrcoef(X_test)
print("CORR_MATRIX TRAIN shape: ", correlation_matrix_train.shape)
print("CORR_MATRIX TEST shape: ", correlation_matrix_test.shape)

CORR_MATRIX TRAIN shape:  (129, 129)
CORR_MATRIX TEST shape:  (33, 33)


In [ ]:
#CONVERT ALL TO TENSORS
x_train = torch.from_numpy(X_train.values)
y_train = torch.from_numpy(y_train.values)
x_test = torch.from_numpy(X_test.values)
y_test = torch.from_numpy(y_test.values)

In [ ]:
threshold_edge = 0.95 #threshold to determine if two nodes are connected based on the correlation matrix

In [ ]:
#BUILD EDGE INDEX FOR TRAIN SET
mask = correlation_matrix_train > threshold_edge

#Exclude the diagonal elements (assuming the matrix is square)
np.fill_diagonal(mask, False)

#Get the row and column indices where the mask is True
row_indices, col_indices = np.where(mask)

#Create the edge_index tensor
edge_index_train = torch.tensor([row_indices, col_indices], dtype=torch.int64)

print(edge_index_train.shape)

torch.Size([2, 4402])


<ipython-input-31-2d88378224f8>:11: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  edge_index_train = torch.tensor([row_indices, col_indices], dtype=torch.int64)


In [ ]:
#BUILD EDGE INDEX FOR TEST SET
mask = correlation_matrix_test > threshold_edge

#Exclude the diagonal elements (assuming the matrix is square)
np.fill_diagonal(mask, False)

#Get the row and column indices where the mask is True
row_indices, col_indices = np.where(mask)

#Create the edge_index tensor
edge_index_test = torch.tensor([row_indices, col_indices], dtype=torch.int64)

print(edge_index_test.shape)

torch.Size([2, 174])


In [ ]:
x_train = x_train.double().to(device)
edge_index_train = edge_index_train.to(device)
y_train = y_train.to(device)
x_test = x_test.double().to(device)
edge_index_test = edge_index_test.to(device)
y_test = y_test.to(device)

In [ ]:
#TRAIN SETTINGS
import torch.optim as optim
import torch.nn as nn

#estimator = GCNEstimator(edge_index=edge_index_train, n_classes=2)
#estimator.fit(x_train, y_train)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 300

In [ ]:
import numpy as np

#TRAIN LOOP
for epoch in range(n_epochs):
    optimizer.zero_grad()
    out = model(x_train.double(), edge_index_train)
    loss = loss_fn(out, y_train)
    # backward pass
    loss.backward()
    if epoch % 10 == 0:
      print(f"EPOCH {epoch}, LOSS: {loss}")
    # update weights
    optimizer.step()

EPOCH 0, LOSS: 0.35330732168290613
EPOCH 10, LOSS: 0.3911746433272572
EPOCH 20, LOSS: 0.43157958974915084
EPOCH 30, LOSS: 0.3498585273447087
EPOCH 40, LOSS: 0.36668370111903664
EPOCH 50, LOSS: 0.3355219697451103
EPOCH 60, LOSS: 0.32314390231197876
EPOCH 70, LOSS: 0.30670811320686964
EPOCH 80, LOSS: 0.2959576718878173
EPOCH 90, LOSS: 0.28693914227736017
EPOCH 100, LOSS: 0.22494909509998612
EPOCH 110, LOSS: 0.25194952667549997
EPOCH 120, LOSS: 0.25267632032196763
EPOCH 130, LOSS: 0.2590382400271774
EPOCH 140, LOSS: 0.20048250463957965
EPOCH 150, LOSS: 0.18952396422940798
EPOCH 160, LOSS: 0.1636221260586394
EPOCH 170, LOSS: 0.15332085169356718
EPOCH 180, LOSS: 0.14402365906097173
EPOCH 190, LOSS: 0.1357858902165787
EPOCH 200, LOSS: 0.12910920326257455
EPOCH 210, LOSS: 0.12281949125158671
EPOCH 220, LOSS: 0.1172621770706821
EPOCH 230, LOSS: 0.1131599273550107
EPOCH 240, LOSS: 0.11429036030902366
EPOCH 250, LOSS: 0.1035822305782646
EPOCH 260, LOSS: 0.09972450151269362
EPOCH 270, LOSS: 0.098

In [ ]:
#TEST
model.eval()
out = model(x_test, edge_index_test)
pred = out.argmax(dim=1)  #Use the class with highest probability.
print(pred)
print(y_test)
test_correct = pred == y_test
test_acc = int(test_correct.sum()) / len(test_correct)
print("Accuracy on test set: {}".format(test_acc))

tensor([0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0,
        0, 1, 1, 1, 1, 0, 1, 1, 1])
tensor([0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0,
        1, 1, 1, 1, 1, 0, 1, 1, 1])
Accuracy on test set: 0.9393939393939394


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/BioInfo/model_weights_meth_most_least_1024.pth')

#Evaluating tumor modified

In [ ]:
connections = pd.read_csv('/content/drive/MyDrive/lung/methylation_gene_connections.csv')
connections

,gene_name,Name,gene_id
0,A1BG,cg22568540,ENSG00000121410.12
1,A2M,cg12058490,ENSG00000175899.15
2,A2ML1,cg03490200,ENSG00000166535.20
3,A2ML1,cg27653134,ENSG00000166535.20
4,A4GALT,cg07393322,ENSG00000128274.17
...,...,...,...
21429,ZYX,cg03100196,ENSG00000159840.16
21430,ZZEF1,cg05922057,ENSG00000074755.15
21431,ZZEF1,cg07607921,ENSG00000074755.15
21432,ZZZ3,cg04127303,ENSG00000036549.13


In [ ]:
most_expressed = pd.read_csv('/content/drive/MyDrive/lung/most_expressed.csv')
most_expressed

,gene_name,node_id
0,ENSG00000169469.9,12385
1,ENSG00000170373.8,12580
2,ENSG00000168530.16,12175
3,ENSG00000274618.2,19383
4,ENSG00000168484.12,12160
...,...,...
7499,ENSG00000184828.10,15157
7500,ENSG00000203747.12,17010
7501,ENSG00000135318.12,6955
7502,ENSG00000203722.8,17002


In [ ]:
least_expressed = pd.read_csv('/content/drive/MyDrive/lung/least_expressed.csv')
least_expressed

,gene_name
0,ENSG00000197497.11
1,ENSG00000196735.13
2,ENSG00000106633.17
3,ENSG00000129911.9
4,ENSG00000196860.8
...,...
12453,ENSG00000283977.1
12454,ENSG00000283967.1
12455,ENSG00000283952.1
12456,ENSG00000256453.2


In [ ]:
cols = most_expressed[:1024]['gene_name'].values
len(cols)

1024

In [ ]:
a = set(cols)
b = set(connections['gene_id'])
cols_final = a.intersection(b)
len(cols_final)

561

In [ ]:
conn = connections[connections['gene_id'].isin(cols_final)]
conn

,gene_name,Name,gene_id
13,AADACL2,cg25159668,ENSG00000197953.6
45,ABCA8,cg21660392,ENSG00000141338.14
159,ACADL,cg09068528,ENSG00000115361.8
160,ACADL,cg14795968,ENSG00000115361.8
257,ACTL7A,cg00065385,ENSG00000187003.7
...,...,...,...
20834,ZFP42,cg06274159,ENSG00000179059.10
20835,ZFP42,cg14189571,ENSG00000179059.10
20863,ZIC2,cg16584172,ENSG00000043355.12
20864,ZIC2,cg24623271,ENSG00000043355.12


In [ ]:
a = set(conn['Name'].values)
col = a.intersection(c)
len(col)

874

In [ ]:
len(c)

1220

In [ ]:
met_final = met_normal.loc[:, c]
met_final.loc[:, col] = met_tumor.loc[:, col]
met_final

<ipython-input-228-5ec7eb3932f7>:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  met_final = met_normal.loc[:, c]
<ipython-input-228-5ec7eb3932f7>:2: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  met_final.loc[:, col] = met_tumor.loc[:, col]
<ipython-input-228-5ec7eb3932f7>:2: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  met_final.loc[:, col] = met_tumor.loc[:, col]
<ipython-input-228-5ec7eb3932f7>:2: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  met_final.loc[:, col] = met_tumor.loc[:, col]


,cg15820189,cg20098478,cg13700897,cg24115040,cg17720231,cg12914014,cg19815720,cg18705776,cg13436799,cg09827833,...,cg25422943,cg19070338,cg11009596,cg16958189,cg23067535,cg01500140,cg03310469,cg02332537,cg01669948,cg12041387
0,0.016556,0.650424,0.068514,0.104426,0.174120,0.324892,0.682927,0.756480,0.205412,0.188374,...,0.254836,0.057490,0.087801,0.231311,0.678587,0.809223,0.104737,0.657064,0.719965,0.209009
1,0.022528,0.418507,0.061249,0.055639,0.077161,0.350191,0.523374,0.645940,0.045887,0.308283,...,0.059583,0.059883,0.073472,0.193691,0.777193,0.790665,0.118517,0.554339,0.407038,0.063230
2,0.019667,0.756253,0.056913,0.209115,0.080279,0.281947,0.689818,0.724767,0.070499,0.300567,...,0.094769,0.065482,0.084706,0.178624,0.581734,0.785989,0.117188,0.720672,0.736992,0.260993
3,0.021161,0.741134,0.057062,0.384706,0.052942,0.315928,0.717875,0.720247,0.049084,0.197905,...,0.052904,0.058321,0.072922,0.221697,0.538774,0.671457,0.123204,0.745862,0.711420,0.443517
4,0.022284,0.864337,0.050416,0.087844,0.207299,0.327785,0.730840,0.926317,0.036576,0.131206,...,0.225360,0.047975,0.074801,0.189122,0.418112,0.795204,0.053597,0.573514,0.434200,0.795184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,0.024168,0.809364,0.039030,0.127789,0.089194,0.282047,0.449717,0.509036,0.048459,0.592684,...,0.071171,0.036358,0.068939,0.183689,0.503290,0.588169,0.077468,0.646641,0.433190,0.281738
77,0.020612,0.770172,0.060090,0.107273,0.115195,0.287378,0.747088,0.801457,0.041629,0.358204,...,0.073691,0.076831,0.052894,0.176298,0.551248,0.822475,0.076741,0.689861,0.699539,0.146358
78,0.019273,0.776077,0.106839,0.215330,0.105762,0.294203,0.702559,0.817139,0.034846,0.553147,...,0.145549,0.056262,0.056304,0.180214,0.673977,0.838915,0.098806,0.697675,0.528442,0.443702
79,0.027155,0.881142,0.074981,0.777086,0.041069,0.369058,0.775247,0.863406,0.032301,0.700372,...,0.075597,0.061877,0.086297,0.225061,0.311019,0.740411,0.070959,0.647229,0.439692,0.701733


In [ ]:
'tumor' in met_final.columns.values

True

In [ ]:
y_test = np.ones(met_final.shape[0])
y_test = torch.from_numpy(y_test).to(device)

In [ ]:
X_test = met_final.drop('tumor', axis=1).values
x_test = torch.from_numpy(X_test).double().to(device)

In [ ]:
correlation_matrix_test = np.corrcoef(X_test)
print("CORR_MATRIX TEST shape: ", correlation_matrix_test.shape)

CORR_MATRIX TEST shape:  (81, 81)


In [ ]:
threshold_edge = 0.95 #threshold to determine if two nodes are connected based on the correlation matrix

In [ ]:
#BUILD EDGE INDEX FOR TEST SET
mask = correlation_matrix_test > threshold_edge

#Exclude the diagonal elements (assuming the matrix is square)
np.fill_diagonal(mask, False)

#Get the row and column indices where the mask is True
row_indices, col_indices = np.where(mask)

#Create the edge_index tensor
edge_index_test = torch.tensor([row_indices, col_indices], dtype=torch.int64)

print(edge_index_test.shape)

torch.Size([2, 202])


In [ ]:
x_test = x_test.double().to(device)
edge_index_test = edge_index_test.to(device)
y_test = y_test.to(device)

In [ ]:
#TEST
model.eval()
out = model(x_test, edge_index_test)
pred = out.argmax(dim=1)  #Use the class with highest probability.
print(pred)
print(y_test)
test_correct = pred == y_test
test_acc = int(test_correct.sum()) / len(test_correct)
print("Accuracy on test set: {}".format(test_acc))

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=torch.float64)
Accuracy on test set: 1.0


#Evaluating with pca

In [ ]:
X = met.drop('tumor', axis=1)
y = met['tumor']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
print("TRAIN SHAPE: ", X_train.shape)
print("TEST SHAPE: ", X_test.shape)
print("NORMAL TRAIN: ", y_train[y_train==0].shape)
print("NORMAL Test: ", y_test[y_test==0].shape)

TRAIN SHAPE:  (786, 19027)
TEST SHAPE:  (197, 19027)
NORMAL TRAIN:  (66,)
NORMAL Test:  (15,)


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=500)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
print("PCA TRAIN shape: ", X_train.shape)
print("PCA TEST shape: ", X_test.shape)

PCA TRAIN shape:  (786, 500)
PCA TEST shape:  (197, 500)


In [ ]:
features_in = X_train.shape[1]
n_classes = len(y_train.unique())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        torch.manual_seed(1234)
        self.conv1 = GCNConv(features_in, 512)
        self.conv2 = GCNConv(512, 256)
        self.conv3 = GCNConv(256, 2)
        self.classifier = Linear(2, n_classes)

    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index)
        h = h.tanh()
        h = self.conv2(h, edge_index)
        h = h.tanh()
        h = self.conv3(h, edge_index)
        h = h.tanh()

        out = self.classifier(h)

        return out

model = GCN().double().to(device)
print(model)

GCN(
  (conv1): GCNConv(500, 512)
  (conv2): GCNConv(512, 256)
  (conv3): GCNConv(256, 2)
  (classifier): Linear(in_features=2, out_features=2, bias=True)
)


In [ ]:
#BUILD CORRELATION MATRIXES
correlation_matrix_train = np.corrcoef(X_train)
correlation_matrix_test = np.corrcoef(X_test)
print("CORR_MATRIX TRAIN shape: ", correlation_matrix_train.shape)
print("CORR_MATRIX TEST shape: ", correlation_matrix_test.shape)

CORR_MATRIX TRAIN shape:  (786, 786)
CORR_MATRIX TEST shape:  (197, 197)


In [ ]:
#CONVERT ALL TO TENSORS
x_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(y_train.values)
x_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test.values)

In [ ]:
threshold_edge = 0.95 #threshold to determine if two nodes are connected based on the correlation matrix

In [ ]:
#BUILD EDGE INDEX FOR TRAIN SET
mask = correlation_matrix_train > threshold_edge

#Exclude the diagonal elements (assuming the matrix is square)
np.fill_diagonal(mask, False)

#Get the row and column indices where the mask is True
row_indices, col_indices = np.where(mask)

#Create the edge_index tensor
edge_index_train = torch.tensor([row_indices, col_indices], dtype=torch.int64)

print(edge_index_train.shape)

torch.Size([2, 3858])


In [ ]:
#BUILD EDGE INDEX FOR TEST SET
mask = correlation_matrix_test > threshold_edge

#Exclude the diagonal elements (assuming the matrix is square)
np.fill_diagonal(mask, False)

#Get the row and column indices where the mask is True
row_indices, col_indices = np.where(mask)

#Create the edge_index tensor
edge_index_test = torch.tensor([row_indices, col_indices], dtype=torch.int64)

print(edge_index_test.shape)

torch.Size([2, 226])


In [ ]:
x_train = x_train.double().to(device)
edge_index_train = edge_index_train.to(device)
y_train = y_train.to(device)
x_test = x_test.double().to(device)
edge_index_test = edge_index_test.to(device)
y_test = y_test.to(device)

In [ ]:
#TRAIN SETTINGS
import torch.optim as optim
import torch.nn as nn

#estimator = GCNEstimator(edge_index=edge_index_train, n_classes=2)
#estimator.fit(x_train, y_train)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 250

In [ ]:
import numpy as np

#TRAIN LOOP
for epoch in range(n_epochs):
    optimizer.zero_grad()
    out = model(x_train.double(), edge_index_train)
    loss = loss_fn(out, y_train)
    # backward pass
    loss.backward()
    if epoch % 10 == 0:
      print(f"EPOCH {epoch}, LOSS: {loss}")
    # update weights
    optimizer.step()

EPOCH 0, LOSS: 0.19552679542186888
EPOCH 10, LOSS: 0.1942239200230779
EPOCH 20, LOSS: 0.19293625467489778
EPOCH 30, LOSS: 0.1916637020296397
EPOCH 40, LOSS: 0.19040619649172583
EPOCH 50, LOSS: 0.18916369855804815
EPOCH 60, LOSS: 0.18793618974114706
EPOCH 70, LOSS: 0.18672366803135
EPOCH 80, LOSS: 0.1855261438570283
EPOCH 90, LOSS: 0.1843436365048424
EPOCH 100, LOSS: 0.18317617096428837
EPOCH 110, LOSS: 0.18202377516307394
EPOCH 120, LOSS: 0.18088647756187634
EPOCH 130, LOSS: 0.17976430507889138
EPOCH 140, LOSS: 0.17865728131630884
EPOCH 150, LOSS: 0.17756542506245315
EPOCH 160, LOSS: 0.17648874904484543
EPOCH 170, LOSS: 0.17542725891087732
EPOCH 180, LOSS: 0.17438095241415566
EPOCH 190, LOSS: 0.1733498187858944
EPOCH 200, LOSS: 0.1723338382719996
EPOCH 210, LOSS: 0.17133298181772236
EPOCH 220, LOSS: 0.1703472108829488
EPOCH 230, LOSS: 0.1693764773723575
EPOCH 240, LOSS: 0.1684207236658013


In [ ]:
#TEST
model.eval()
out = model(x_test, edge_index_test)
pred = out.argmax(dim=1)  #Use the class with highest probability.
print(pred)
print(y_test)
test_correct = pred == y_test
test_acc = int(test_correct.sum()) / len(test_correct)
print("Accuracy on test set: {}".format(test_acc))

tensor([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
        1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1])
tensor([1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1,